In [1]:
import os
# Install java
#! sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

os.environ["SPARK_HOME"] = "/opt/conda/lib/python3.7/site-packages/pyspark/"
os.environ["PATH"] = os.environ["SPARK_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["geospark.global.charset"]="utf8"


In [2]:
import sparknlp
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, to_timestamp,date_format
from pyspark import StorageLevel
import pyspark.sql.functions as F
from sparknlp.pretrained import PretrainedPipeline
from collections import Counter
from wordcloud import WordCloud
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
from shapely.geometry import Point
from shapely.geometry import Polygon

from geospark.register import GeoSparkRegistrator
from geospark.register import upload_jars
from geospark.core.SpatialRDD import SpatialRDD
from geospark.core.SpatialRDD import PointRDD
from geospark.core.SpatialRDD import PolygonRDD
from geospark.core.SpatialRDD import LineStringRDD
from geospark.core.enums import FileDataSplitter
from geospark.utils.adapter import Adapter
from geospark.core.spatialOperator import KNNQuery
from geospark.core.spatialOperator import JoinQuery
from geospark.core.spatialOperator import RangeQuery
from geospark.core.formatMapper.shapefileParser import ShapefileReader
from geospark.core.formatMapper import WkbReader
from geospark.core.formatMapper import WktReader
from geospark.core.formatMapper import GeoJsonReader
from geospark.sql.types import GeometryType
from geospark.core.enums import GridType
from geospark.core.SpatialRDD import RectangleRDD
from geospark.core.enums import IndexType
from geospark.core.geom.envelope import Envelope
from geospark.utils import GeoSparkKryoRegistrator, KryoSerializer

def start():
    builder = SparkSession.builder \
        .appName("Spark-Geo-Spatial") \
        .master("local[*]") \
        .config("spark.driver.memory", "24G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryo.registrator", GeoSparkKryoRegistrator.getName) \
        .config("spark.kryoserializer.buffer.max", "2040M") \
        .config("geospark.global.charset", "utf8")
    return builder.getOrCreate()
spark = start()


In [3]:
GeoSparkRegistrator.registerAll(spark)

True

In [4]:
upload_jars()

True

In [5]:
import geopandas as gpd
geo_json_rdd = GeoJsonReader.readToGeometryRDD(spark.sparkContext, "outbreak1.geojson",False, True)

In [ ]:
df = geo_json_rdd.toPandas()
gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf.plot(
    figsize=(10, 8),
    column="value",
    legend=True,
    cmap='YlOrBr',
    scheme='quantiles',
    edgecolor='lightgray'
)

In [ ]:
df = Adapter.toDf(geo_json_rdd, spark).show(5, True)